In [55]:
# Imports
import pandas as pd
from zipfile import ZipFile
from io import BytesIO
import urllib.request as urllib2
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from datetime import datetime, timedelta

In [3]:
## Set up base url for citibike data
base_url = "https://s3.amazonaws.com/tripdata/"

## Gets url endpoints for each individual dataset in 2022, and the names of the associated csv files
datasets = {}
for i in range(1, 13):
    datasets[i] = {}
    # Accounts for a typo in the file
    if i == 6 or i == 7:
        datasets[i]['zip'] = f"20220{i}-citbike-tripdata.csv.zip"
        datasets[i]['csv'] = f"20220{i}-citbike-tripdata.csv"
    elif i < 10:
        datasets[i]['zip'] = f"20220{i}-citibike-tripdata.csv.zip"
        datasets[i]['csv'] = f"20220{i}-citibike-tripdata.csv"
    else:
        datasets[i]['zip'] = f"2022{i}-citibike-tripdata.csv.zip"
        datasets[i]['csv'] = f"2022{i}-citibike-tripdata.csv"

In [4]:
## Read each dataset into a DataFrame
# Sets up a list for the dataframes
data = []

# Loops through the urls in the datasets dictionary 
for i in range(1, 13):
    # Reads the resulting zip files
    read = urllib2.urlopen(base_url+datasets[i]['zip']).read()
    # Reads the zip files into a BytesIO file
    file = ZipFile(BytesIO(read))
    # Creates a dataframe from the csv from the zip file and puts it into the data dictionary
    data.append(pd.read_csv(file.open(datasets[i]['csv'])))

# Concatenates all dataframes together
all_data = pd.concat(data)

# Deletes the data list to free memory
del data

C:\Users\kronh\AppData\Local\Temp\ipykernel_22276\1795321811.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data.append(pd.read_csv(file.open(datasets[i]['csv'])))
C:\Users\kronh\AppData\Local\Temp\ipykernel_22276\1795321811.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data.append(pd.read_csv(file.open(datasets[i]['csv'])))
C:\Users\kronh\AppData\Local\Temp\ipykernel_22276\1795321811.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data.append(pd.read_csv(file.open(datasets[i]['csv'])))
C:\Users\kronh\AppData\Local\Temp\ipykernel_22276\1795321811.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data.append(pd.read_csv(file.open(datasets[i]['csv'])))
C:\Users\kronh\AppData\Local\Temp\ipykernel_22276\1795321811.py:12: DtypeWarning: Column

In [61]:
# Drops all rows with a null value
all_data = all_data.dropna()

# Dictionary to change the data types of started_at and ended_at columns in the dataframe
type_conversion = {
    "started_at": "datetime64",
    "ended_at": "datetime64"
}

# Changes the types of the specified columns
all_data = all_data.astype(type_conversion)

# Filters out the rows where the 'started_at' time is earlier than the 'ended_at' time (since this is presumably incorrect or flawed data)
all_data = all_data[all_data["started_at"] < all_data['ended_at']]

# Takes a 15 million row random sample of the data (since Tableau public limits datasets to 15 mil rows)
sampled_data = all_data.sample(n=15000000, random_state=1, axis=0, ignore_index=True)

# Calculates the length of each trip and sets that to a new column
sampled_data['trip_duration'] = sampled_data['ended_at'] - sampled_data['started_at']

# Converts the trip_duration column to total seconds
sampled_data['trip_duration'] = sampled_data['trip_duration'].dt.total_seconds().astype(int)

# Creates a column for path ID
sampled_data["trip"] = sampled_data['start_station_name'] + " / " + sampled_data['end_station_name']

# Shows the final DataFrame
sampled_data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,trip
0,10C56BB94458C785,classic_bike,2022-09-29 07:55:29,2022-09-29 07:59:46,E 12 St & 3 Ave,5788.12,W 13 St & 7 Ave,6030.04,40.732233,-73.988900,40.737815,-73.999947,casual,257,E 12 St & 3 Ave / W 13 St & 7 Ave
1,F830AFB154C07A5F,classic_bike,2022-08-17 13:01:39,2022-08-17 13:15:44,Canal St & Rutgers St,5303.08,University Pl & E 14 St,5905.14,40.714275,-73.989900,40.734814,-73.992085,member,845,Canal St & Rutgers St / University Pl & E 14 St
2,B0C087BA020313D8,electric_bike,2022-08-27 13:23:07,2022-08-27 13:28:33,E 14 St & Avenue B,5736.09,St Marks Pl & 2 Ave,5669.10,40.729387,-73.977724,40.728419,-73.987140,member,326,E 14 St & Avenue B / St Marks Pl & 2 Ave
3,8B96A3B9A67C666A,classic_bike,2022-11-04 16:46:02,2022-11-04 17:11:38,Vernon Blvd & 50 Ave,6170.02,Montrose Ave & Bushwick Ave,5068.02,40.742327,-73.954117,40.707678,-73.940162,member,1536,Vernon Blvd & 50 Ave / Montrose Ave & Bushwick...
4,6EBDB47F431CA62B,electric_bike,2022-03-24 20:44:05,2022-03-24 20:51:33,Henry St & Grand St,5294.04,St James Pl & Pearl St,5167.06,40.714211,-73.981095,40.711174,-74.000165,member,448,Henry St & Grand St / St James Pl & Pearl St
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,9AE321826CEA3588,electric_bike,2022-09-07 18:26:29,2022-09-07 18:36:46,N 11 St & Kent Ave,5489.04,Graham Ave & Herbert St,5403.05,40.722482,-73.959219,40.719293,-73.945004,member,617,N 11 St & Kent Ave / Graham Ave & Herbert St
14999996,C73D4AB9401A4B53,classic_bike,2022-02-06 17:14:36,2022-02-06 17:44:23,24 St & 5 Ave,3555.04,Fulton St & Utica Ave,4228.02,40.659555,-73.995068,40.679427,-73.929891,member,1787,24 St & 5 Ave / Fulton St & Utica Ave
14999997,F39C12D578D9DF80,classic_bike,2022-05-28 17:05:38,2022-05-28 17:26:07,7 Ave S & Bleecker St,5805.07,Broadway & W 29 St,6289.06,40.732139,-74.003638,40.746201,-73.988557,casual,1229,7 Ave S & Bleecker St / Broadway & W 29 St
14999998,5010273D49CDEDC7,docked_bike,2022-07-21 16:36:25,2022-07-21 16:52:33,12 Ave & W 40 St,6765.01,W 37 St & 5 Ave,6398.06,40.760875,-74.002777,40.750380,-73.983390,casual,968,12 Ave & W 40 St / W 37 St & 5 Ave


In [60]:
# Exports the sampled dataframe to a csv file just OUTSIDE of the repository
sampled_data.to_csv("../../../citibike_2022_data.csv", index=False)

In [50]:
## Get zip code and neghborhood information for each end station

# Sets up a system to find address from latitude/ longitude pairs
find_address = RateLimiter(Nominatim(user_agent="citibike_location_finder").reverse, min_delay_seconds=1, max_retries=0)

# Create a new dataframe with each unique end station name that includes station name, station id, latitude, and longitude
stations = sampled_data[["end_station_name", "end_lat", "end_lng"]].drop_duplicates("end_station_name").reset_index(drop=True)

# Adds columns for zipcode from the geolocator
stations["zipcode"] = ""

# Find address information for each lat/ lng pair and add zipcode to the stations DataFrame
# Locations that aren't associated with a postal code are printed below and their zipcode rows are empty
for index, row in stations.iterrows():
    temp = find_address(f"{row['end_lat']}, {row['end_lng']}").raw
    try:
        stations.at[index, 'zipcode'] = temp['address']['postcode']
    except:
        print(f"There was an error at: {row[['end_station_name', 'end_lat', 'end_lng']]}")

There was an error at: end_station_name    58 St & Pier 4
end_lat                  40.646475
end_lng                 -74.026081
Name: 130, dtype: object
There was an error at: end_station_name    Stagg St & Union Ave
end_lat                        40.708771
end_lng                       -73.950953
Name: 132, dtype: object
There was an error at: end_station_name    E 53 St & Lexington Ave
end_lat                           40.758281
end_lng                          -73.970694
Name: 218, dtype: object
There was an error at: end_station_name    Cooper Square & Astor Pl
end_lat                            40.729515
end_lng                           -73.990753
Name: 231, dtype: object
There was an error at: end_station_name    W 52 St & 6 Ave
end_lat                    40.76133
end_lng                   -73.97982
Name: 539, dtype: object
There was an error at: end_station_name    Sunken Meadow Comfort Station
end_lat                                  40.79558
end_lng                           

In [54]:
# Remove lat and lng from stations dataframe
stations = stations[['end_station_name', 'zipcode']]

# Rename end_station_name
stations.rename(columns={"end_station_name": "station_name"})

# Writes the names and zipcodes to a csv file in the repository
stations.to_csv("station_zipcodes.csv", index=False)
